In [95]:
from pathlib import Path 
import os
import pandas as pd
from collections import OrderedDict
import numpy as np
import sys
from sklearn.metrics.pairwise import cosine_similarity

#sys.path.append("/Users/alina/Desktop/MIT/code/ADHD/item_analysis")
from src import topic_modeling as tm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [83]:
data_root = Path("/Volumes/Samsung_T5/MIT/abcd/release_05/abcd-data-release-5.1")
vocab_path  = Path("/Volumes/Samsung_T5/MIT/abcd/adhd_vocab.npy")

section = "core/mental-health"
table = 'mh_p_ksads_adhd.csv'
mapping = 'ksads_adhd.csv'

data_section = Path( data_root, section)
data_path = Path(data_section, table) # path to results table  
var_mapping_path =  Path(data_root, "var_mapping", mapping) # path to mapping of variable names to questions 

#os.listdir(data_section)

In [36]:
abbreviation = {"mental-health" : "mp", "parent": "p", "youth": "y"}
event_names = [ "baseline_year_1_arm_1","1_year_follow_up_y_arm_1","2_year_follow_up_y_arm_1","3_year_follow_up_y_arm_1", "4_year_follow_up_y_arm_1", "4_year_follow_up_y_arm_1" ]
year = dict(zip(range(0, 6), event_names)) # asses year in the column "eventname"

In [85]:
data = pd.read_csv(data_path)
var_description = pd.read_csv(var_mapping_path)
vocab = np.load(vocab_path, allow_pickle=True)

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_13831/2715051642.py:1: DtypeWarning: Columns (23,76,83,84) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(data_path)


In [87]:
var_description.head()

,table_name,var_name,var_label,notes,condition,table_name_nda
0,mh_p_ksads_adhd,ksads_adhd_raw_978_p,"In the past two weeks, how often has your chil...",0=Not at all; 1=Rarely; 2=Several days; 3=More...,NaN,attn_deficit_hyperactiv_p01
1,mh_p_ksads_adhd,ksads_adhd_raw_991_p,Do the problems we have just been talking abou...,0=No; 1=Yes,NaN,attn_deficit_hyperactiv_p01
2,mh_p_ksads_adhd,ksads_adhd_raw_990_p,Do the problems we have just been talking abou...,0=No; 1=Yes,NaN,attn_deficit_hyperactiv_p01
3,mh_p_ksads_adhd,ksads_adhd_raw_988_p,"In the past two weeks, how often has your chil...",0=Not at all; 1=Rarely; 2=Several days; 3=More...,NaN,attn_deficit_hyperactiv_p01
4,mh_p_ksads_adhd,ksads_adhd_raw_987_p,"In the past two weeks, how often has your chil...",0=Not at all; 1=Rarely; 2=Several days; 3=More...,NaN,attn_deficit_hyperactiv_p01


In [70]:
var_names = np.array(var_description["var_name"])
questions = np.array(var_description["var_label"])
mapping = OrderedDict(zip(var_names, questions))

In [73]:
questions

array(['In the past two weeks, how often has your child put off or avoided doing things that were hard for him or her because they required a lot of attention?',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With friends',
       "In the past two weeks, how often has your child talked when he or she wasn't supposed to, or had people complain that he or she talked too much?",
       'In the past two weeks, how often has your child interrupted someone when they were talking?',
       'In the past two weeks, how often has your child had trouble waiting for his or her turn or waiting for other things?',
       "In the past two weeks, how often has your child blurt out answers in school without being called on, or blurt out answers when someone wasn't talking to him or her?",
       'In

In [88]:
embeddings_qst = tm.get_embeddings(questions, transformer='dmis-lab/biobert-base-cased-v1.2') # 'distilbert-base-nli-mean-tokens'
embeddings_vocab = tm.get_embeddings(vocab, transformer='dmis-lab/biobert-base-cased-v1.2')

No sentence-transformers model found with name dmis-lab/biobert-base-cased-v1.2. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

No sentence-transformers model found with name dmis-lab/biobert-base-cased-v1.2. Creating a new one with MEAN pooling.


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [94]:
embeddings_qst.shape

(25, 768)

In [98]:
similarity = cosine_similarity(embeddings_qst, embeddings_vocab)

In [111]:
questions[np.where(similarity == max(similarity[1]))[0]]
#vocab[np.where(similarity == max(similarity[1]))[1]]

array(['Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family',
       'Do the problems we have just been talking about cause difficulties for your child in any of the following areas:With family'],
      dtype=object)